## Coding Exercise #0708b

### 1. Inverse transform with AutoEncoder (reduced dimensional input):

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data            # MNIST handwritten digits data!
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline

#### 1.1. Download the MNIST data:

In [ ]:
# verbosity_saved = tf.logging.get_verbosity()                        # Save the current verbosity lebel if needed.
tf.logging.set_verbosity(tf.logging.ERROR)                            # Set the verbosity lebel high so that most warnings are ignored. 
mnist = input_data.read_data_sets("MNIST_data/",one_hot=True)         # Download the data.
type(mnist)      

#### 1.2. Visualize the data:

In [ ]:
n_rows = 3
n_cols = 3
n_shift = 10
f, a = plt.subplots(n_rows, n_cols, figsize=(5, 5))
for i in range(n_rows):
    for j in range(n_cols):
        single_image = mnist.test.images[n_shift+i*n_cols+j].reshape(28,28) 
        a[i,j].imshow(1.0-single_image, cmap='gist_gray')      # In Grayscale.
plt.show()

#### 1.3. Define an AutoEncoder model:

In [ ]:
# Hyperparameters definition.
n_input = 784             # Input layers has as many nodes as the number of variables.
n_hidden = 98             # The number of nodes in the hidden layer <= targeted reduced dimension. 
n_output = n_input       # The output should have the same number of nodes as the input. 
learn_rate = 0.005
n_epochs = 1001

In [ ]:
# Variables definition.
initializer = tf.variance_scaling_initializer()
W1 = tf.Variable(initializer([n_input, n_hidden]), dtype=tf.float32)
W2 = tf.Variable(initializer([n_hidden, n_output]), dtype=tf.float32)
b1 = tf.Variable(tf.zeros(n_hidden))
b2 = tf.Variable(tf.zeros(n_output))

In [ ]:
# Placeholder definition.
X_ph = tf.placeholder(tf.float32, shape=[None, n_input])

In [ ]:
# AutoEncoder model definition.
hidden_layer = tf.matmul(X_ph, W1) + b1                  # ReLu activation.
y_model = tf.matmul(hidden_layer, W2) + b2

In [ ]:
loss = tf.reduce_mean(tf.square(X_ph- y_model))         # 'X' takes the place of 'y'!!!

In [ ]:
optimizer = tf.train.AdamOptimizer(learning_rate = learn_rate) 

In [ ]:
train = optimizer.minimize(loss)

In [ ]:
init = tf.global_variables_initializer()

#### 1.4. AutoEncoder training:

In [ ]:
saver = tf.train.Saver() 
with tf.Session() as sess:
        sess.run(init)
        my_feed = {X_ph: mnist.test.images}
        for i in range(n_epochs):
            sess.run(train, feed_dict = my_feed)
            mse = sess.run(loss, feed_dict = my_feed)
            if (i % 100 == 0):
                print("Step : {}    ,    MSE : {}".format(i, mse))
        saver.save(sess, "./autoencoder")           # Save the result.

#### 1.5. Visualize the result:

In [ ]:
# Left 3x3 are the original.
# Right 3x3 are the reduced dimensional input.
n_images = n_rows*n_cols
with tf.Session() as sess:
    saver.restore(sess,"./autoencoder")
    X_inverse_transform = y_model.eval(feed_dict={X_ph:mnist.test.images[n_shift:n_shift+n_images]})      # The output is the reduced dimensional input!
    f, a = plt.subplots(n_rows, 2*n_cols, figsize=(20, 10))
    for i in range(n_rows):
        for j in range(n_cols):
            single_image1 = mnist.test.images[n_shift+i*n_cols+j].reshape(28,28) 
            single_image2 = X_inverse_transform[i*n_cols+j].reshape(28,28) 
            a[i][j].imshow(1.0-single_image1, cmap='gist_gray')             # Grayscale.
            a[i][j+n_cols].imshow(1.0-single_image2, cmap='gist_gray')      # Grayscale.
    plt.show()